In [ ]:
#Exponential with Qs as not static
import queue as Q
import numpy as np
import random
import matplotlib.pyplot as plt

#import bpy

k = 1000
p = 10**10
alpha = 1.5

random.seed(15)

def generate_random_point(low, high):
    random.seed(15)
    xy = np.random.uniform(low, high, (1,1))
    return xy

def generate_random_bp(k, p, alpha):
    random.seed(15)
    pt = generate_random_point(0,1)
    u = pt[0,0]
    x = k/(1+ u*((k/p)**alpha - 1))**(1/alpha)
    return x
    
class Event(object):
    def __init__(self, eventTime, eventType, eventID):
        self.eventTime = eventTime
        self.eventType = eventType
        self.eventID = eventID
        self.eventServiceTime = 0.0
        self.eventWaitingTime = 0.0

        return
    
    def __le__(self, other):
        return ((self.eventTime < other.eventTime) or (self.eventTime == other.eventTime))
    def __ge__(self, other):
        return ((self.eventTime > other.eventTime) or (self.eventTime == other.eventTime))
    def __lt__(self, other):
        return (self.eventTime < other.eventTime)
    def __gt__(self, other):
        return (self.eventTime > other.eventTime)
    def __eq__(self, other):
        return (self.eventTime == other.eventTime)
    def __ne__(self, other):
        return (self.eventTime != other.eventTime)
    
    def setTime(self, etime):
        self.eventTime = etime
    def setType(self, etype):
        self.eventType = etype     
    def setID(self, eID):
        self.eventTime = eID
    def setServiceTime(self, eServiceTime):
        self.eventTime = eServiceTime
    def setWaitingTime(self, eWaitingTime):
        self.eventTime = eWaitingTime
    def getTime(self):
        return self.eventTime
    def getType(self):
        return self.eventType
    def getID(self):
        return self.eventID
    def getServiceTime(self):
        return self.eventServiceTime
    def getWaitingTime(self):
        return self.eventWaitingTime

       
    
class Sim(object):
    #FutureEventList = Q.PriorityQueue()
    #Jobs1 = Q.Queue()
    #ServiceTimeQ1 = Q.Queue()
    #FutureEventList2 = Q.PriorityQueue()
    #Jobs2 = Q.Queue()
    #ServiceTimeQ2 = Q.Queue()
    #ToBeScheduledNextQ1 = Q.Queue()
    #ToBeScheduledNextQ2 = Q.Queue()
    
    def __init__(self, meanInterArrivalTime, meanServiceTime1, meanServiceTime2):
        print('Printing from initialization')
        self.meanInterArrivalTime = meanInterArrivalTime
        self.meanServiceTime1 = meanServiceTime1
        self.meanServiceTime2 = meanServiceTime2
        self.FutureEventList = Q.PriorityQueue()
        self.Jobs1 = Q.Queue()
        self.ServiceTimeQ1 = Q.Queue()
        self.FutureEventList2 = Q.PriorityQueue()
        self.Jobs2 = Q.Queue()
        self.ServiceTimeQ2 = Q.Queue()
        self.ToBeScheduledNextQ1 = Q.Queue()
        self.ToBeScheduledNextQ2 = Q.Queue()
        self.queueLength1 = 0
        self.queueLength2 = 0
        self.numInService1 = 0
        self.numInService2 = 0
        self.lastEventTime1 = 0.0
        self.lastEventTime2 = 0.0
        self.totalBusy1 = 0.0
        self.totalBusy2 = 0.0
        self.sumResponseTime1 = 0.0
        self.sumResponseTime2 = 0.0
        self.sumWaitingTime1 = 0.0
        self.sumWaitingTime2 = 0.0
        self.numOfDepartures1 = 0
        self.numOfDepartures2 = 0
        self.sumResponseTime1To2Cumulative = 0.0
        self.sumWaitingTime1To2Cumulative = 0.0
        self.numDirect2 = 0
        self.num1To2 = 0
        self.eventIDCounter1 = 0
        self.eventIDCounter2 = 100000
        self.q1q2WaitingTime = {}
        self.q1q2ServiceTime = {}
        self.q2WaitingTime = {}
        self.q2ServiceTime = {}
        


    def initialization(self):
        timeToFirstArrival = self.generateRandomInterArrivalTime()
        arrival1ToBeScheduled = self.checkArrival1OrArrival2()
        if(arrival1ToBeScheduled == True):

            self.eventIDCounter1 = self.eventIDCounter1 + 1
            arrivalEvent = Event(timeToFirstArrival, 'arrival1', self.eventIDCounter1)
        else:

            self.eventIDCounter2 = self.eventIDCounter2 + 1
            arrivalEvent = Event(timeToFirstArrival, 'directArrival2', self.eventIDCounter2) 
        self.FutureEventList.put(arrivalEvent)
        return
    

    def generateRandomInterArrivalTime(self):
        random.seed(15)
        return np.random.exponential(scale=self.meanInterArrivalTime)

    def generateRandomServiceTime1(self):
        random.seed(15)
        return np.random.exponential(scale=self.meanServiceTime1)    
    
    def generateRandomServiceTime2(self):
        random.seed(15)
        return np.random.exponential(scale=self.meanServiceTime2)
    
    def generateRandomServiceTime(self):
        random.seed(15)
        return np.random.exponential(scale=self.meanServiceTime1)

 
    def checkArrival1OrArrival2(self):
        x = np.random.uniform(0, 1, (1,1))
        if(x[0,0] < 0.75):
            return True
        else:
            return False
                
    def processArrival1(self, event, clock):
        self.Jobs1.put(event)
        self.ToBeScheduledNextQ1.put(event)
        self.queueLength1 = self.queueLength1+1
        

        
        if(self.numInService1 == 0):
            nextEvent = self.ToBeScheduledNextQ1.get()
            self.scheduleService1(clock, nextEvent.getID())
        else:
            self.totalBusy1 = self.totalBusy1 + clock - self.lastEventTime1

            
        #Schedule next arrival
        timeToNextArrival = self.generateRandomInterArrivalTime()
        arrival1ToBeScheduled = self.checkArrival1OrArrival2()
        
        if(arrival1ToBeScheduled == True):
            self.eventIDCounter1 = self.eventIDCounter1 + 1
            nextArrival = Event(clock+timeToNextArrival, 'arrival1', self.eventIDCounter1)
        else:
            self.eventIDCounter2 = self.eventIDCounter2 + 1
            nextArrival = Event(clock+timeToNextArrival, 'directArrival2', self.eventIDCounter2)            
        
        self.FutureEventList.put(nextArrival)
        self.lastEventTime1 = clock

        
    def scheduleService1(self, clock, eventID):

        serviceTime = -1.0
        while(serviceTime < 0):
            random.seed(15)
            serviceTime = self.generateRandomServiceTime1()
        nextDeparture = Event(clock+serviceTime, 'departure1', eventID)
        self.FutureEventList.put(nextDeparture)
        self.numInService1 = 1
        self.queueLength1 = self.queueLength1 - 1
        scheduledJob = Event(serviceTime, 'service', eventID)
        self.ServiceTimeQ1.put(scheduledJob)
        
    def processDeparture1(self, event, clock):
        finished1 = self.Jobs1.get()
        serviceDoneJob = self.ServiceTimeQ1.get()
        

        if(self.queueLength1 > 0):
            nextEvent = self.ToBeScheduledNextQ1.get()
            self.scheduleService1(clock, nextEvent.getID())
        else:
            self.numInService1 = 0
            
        responseTime = clock - finished1.getTime()
        self.sumResponseTime1 =  self.sumResponseTime1 + responseTime
        
        serviceTime = serviceDoneJob.getTime()
        waitingTime = responseTime - serviceTime
        self.sumWaitingTime1 = self.sumWaitingTime1 + waitingTime
        

        self.q1q2WaitingTime[event.getID()] = waitingTime
        self.q1q2ServiceTime[event.getID()] = serviceTime
    
        secondStageJob = Event(clock, 'arrival1ToArrival2', event.getID())
        self.FutureEventList.put(secondStageJob)       

        self.totalBusy1 = self.totalBusy1 + clock - self.lastEventTime1
        self.numOfDepartures1 = self.numOfDepartures1 + 1
        self.lastEventTime1 = clock

        
    def processArrival2(self, event, clock):
        self.Jobs2.put(event)
        self.ToBeScheduledNextQ2.put(event)
        self.queueLength2 = self.queueLength2+1
        
        
        if(self.numInService2 == 0):
            nextEvent = self.ToBeScheduledNextQ2.get()
            self.scheduleService2(clock, nextEvent.getID())
        else:
            self.totalBusy2 = self.totalBusy2 + clock - self.lastEventTime2

            
        #Schedule next arrival if this is a direct arrival
        eventType = event.getType()
        if(eventType == 'directArrival2'):
            direct = True
        else:
            direct = False
    
        if(direct == True):
            timeToNextArrival = self.generateRandomInterArrivalTime()
            arrival1ToBeScheduled = self.checkArrival1OrArrival2()
            if(arrival1ToBeScheduled == True):
                self.eventIDCounter1 = self.eventIDCounter1 + 1
                nextArrival = Event(clock+timeToNextArrival, 'arrival1', self.eventIDCounter1)
            else:
                self.eventIDCounter2 = self.eventIDCounter2 + 1
                nextArrival = Event(clock+timeToNextArrival, 'directArrival2', self.eventIDCounter2)    
            
            self.FutureEventList.put(nextArrival)
        self.lastEventTime2 = clock

    def scheduleService2(self, clock, eventID):

        serviceTime = -1.0
        while(serviceTime < 0):
            random.seed(15)
            serviceTime = self.generateRandomServiceTime2()
        nextDeparture = Event(clock+serviceTime, 'departure2', eventID)
        self.FutureEventList.put(nextDeparture)
        self.numInService2 = 1
        self.queueLength2 = self.queueLength2 - 1
        scheduledJob = Event(serviceTime, 'service', eventID)
        self.ServiceTimeQ2.put(scheduledJob)
    
        
    def processDeparture2(self, event, clock):

        finished2 = self.Jobs2.get()
        serviceDoneJob = self.ServiceTimeQ2.get()
       
        if(self.queueLength2 > 0):
            nextEvent = self.ToBeScheduledNextQ2.get()
            self.scheduleService2(clock, nextEvent.getID())
        else:
            self.numInService2 = 0
            
        responseTime = clock - finished2.getTime()
        self.sumResponseTime2 =  self.sumResponseTime2 + responseTime
        
        serviceTime = serviceDoneJob.getTime()
        waitingTime = responseTime - serviceTime
        self.sumWaitingTime2 = self.sumWaitingTime2 + waitingTime
        self.totalBusy2 = self.totalBusy2 + clock - self.lastEventTime2
        self.numOfDepartures2 = self.numOfDepartures2 + 1
        
        if(event.getID() > 100000):
            self.q2WaitingTime[event.getID()] = waitingTime
            self.q2ServiceTime[event.getID()] = serviceTime
            self.numDirect2 = self.numDirect2 + 1
        else:

            self.q1q2WaitingTime[event.getID()] = self.q1q2WaitingTime[event.getID()] + waitingTime
            self.q1q2ServiceTime[event.getID()] = self.q1q2ServiceTime[event.getID()] + serviceTime  
            self.num1To2 = self.num1To2 + 1
        
           
        self.lastEventTime2 = clock

    def generateMeanTimeStats(self):
        i = 0
        numQ1Q2 = 0
        sumQ1Q2WaitingTime = 0.0
        sumQ1Q2ServiceTime = 0.0
        for key, value in self.q1q2WaitingTime.items():
            sumQ1Q2WaitingTime = sumQ1Q2WaitingTime+value
            numQ1Q2 = numQ1Q2+1
        for key, value in self.q1q2ServiceTime.items():
            sumQ1Q2ServiceTime = sumQ1Q2ServiceTime+value

        return [self.sumWaitingTime1/self.num1To2, self.sumWaitingTime2/(self.num1To2+self.numDirect2), sumQ1Q2WaitingTime/numQ1Q2, self.sumResponseTime1/self.num1To2, self.sumResponseTime2/(self.num1To2+self.numDirect2), (sumQ1Q2ServiceTime+sumQ1Q2WaitingTime)/numQ1Q2, self.queueLength1, self.queueLength2]

            
        
    def reportGeneration(self):
        print('Avg Waiting time 1')
        print(self.sumWaitingTime1/self.num1To2)
        print('Avg Waiting time 2')
        print(self.sumWaitingTime2/(self.num1To2+self.numDirect2))
        print('Avg Response time 1')
        print(self.sumResponseTime1/self.num1To2)
        print('Avg Response time 2')
        print(self.sumResponseTime2/(self.num1To2+self.numDirect2))
        [avgQ1Q2WaitingTime, avgQ1Q2ServiceTime] = self.generateMeanTimeStats()
        print('Avg Waiting Time 1+2')
        print(avgQ1Q2WaitingTime)
        print('Avg Service Time 1+2')
        print(avgQ1Q2ServiceTime)
        #print('totalBusy')
        #print(self.totalBusy1 + self.totalBusy2)
        print('Wait queue in 1')
        print(self.queueLength1)
        print('Wait queue in 2')
        print(self.queueLength2)
        #print('QSize')
        #print(Sim.FutureEventList.qsize())
        print('Number of departures from 1')
        print(self.num1To2)
        print('Number of departures from 2')
        print(self.numDirect2)
        return
    
totalCustomers = 10000

avgWaitingTime1List = []
avgWaitingTime2List = []
avgWaitingTimeQ1Q2List = [] 
avgResponseTime1List= [] 
avgResponseTime2List = [] 
avgResponseTimeQ1Q2List = []
q1LengthList = [] 
q2LengthList = []

    
i = 0
while(i < 100):


    simClock = 0.0
    simu = Sim(4000, 3000, 3000)
    simu.initialization()
    while((simu.numOfDepartures2 < totalCustomers)):
        event = simu.FutureEventList.get()
        eventType = event.getType()
        eventTime = event.getTime()
        eventID = event.getID()
        simClock = eventTime
        if(eventType == 'arrival1'):
            simu.processArrival1(event, simClock)
        elif(eventType == 'directArrival2'):
            simu.processArrival2(event, simClock)
        elif(eventType == 'arrival1ToArrival2'):
            simu.processArrival2(event, simClock)
        elif(eventType == 'departure1'):
            simu.processDeparture1(event, simClock)
        else:
            simu.processDeparture2(event, simClock)
        del event
    [avgWaitingTime1, avgWaitingTime2, avgWaitingTimeQ1Q2, avgResponseTime1, avgResponseTime2, avgResponseTimeQ1Q2, q1Length, q2Length] = simu.generateMeanTimeStats()
    print([avgWaitingTime1, avgWaitingTime2, avgWaitingTimeQ1Q2, avgResponseTime1, avgResponseTime2, avgResponseTimeQ1Q2, q1Length, q2Length])  

    i = i+1
    avgWaitingTime1List.append(avgWaitingTime1)
    avgWaitingTime2List.append(avgWaitingTime2)
    avgWaitingTimeQ1Q2List.append(avgWaitingTimeQ1Q2)
    avgResponseTime1List.append(avgResponseTime1)
    avgResponseTime2List.append(avgResponseTime2)
    avgResponseTimeQ1Q2List.append(avgResponseTimeQ1Q2)
    q1LengthList.append(q1Length)
    q2LengthList.append(q2Length)

print('avgWaitingTime1List')
print(avgWaitingTime1List)
print('avgWaitingTime2List')
print(avgWaitingTime2List)
print('avgWaitingTimeQ1Q2List')
print(avgWaitingTimeQ1Q2List)
print('avgResponseTime1List')
print(avgResponseTime1List)
print('avgResponseTime2List')
print(avgResponseTime2List)
print('avgResponseTimeQ1Q2List')
print(avgResponseTimeQ1Q2List)
print('q1LengthList')
print(q1LengthList)
print('q2LengthList')
print(q2LengthList)
plt.hist(q2Length)
plt.show()

In [ ]:
#Bounded Pareto with Qs as not static
import queue as Q
import numpy as np
import random
import matplotlib.pyplot as plt

#import bpy

k = 1000
p = 10**10
alpha = 1.5

random.seed(15)

def generate_random_point(low, high):
    random.seed(15)
    xy = np.random.uniform(low, high, (1,1))
    return xy

def generate_random_bp(k, p, alpha):
    random.seed(15)
    pt = generate_random_point(0,1)
    u = pt[0,0]
    x = k/(1+ u*((k/p)**alpha - 1))**(1/alpha)
    return x
    
class Event(object):
    def __init__(self, eventTime, eventType, eventID):
        self.eventTime = eventTime
        self.eventType = eventType
        self.eventID = eventID
        self.eventServiceTime = 0.0
        self.eventWaitingTime = 0.0

        return
    
    def __le__(self, other):
        return ((self.eventTime < other.eventTime) or (self.eventTime == other.eventTime))
    def __ge__(self, other):
        return ((self.eventTime > other.eventTime) or (self.eventTime == other.eventTime))
    def __lt__(self, other):
        return (self.eventTime < other.eventTime)
    def __gt__(self, other):
        return (self.eventTime > other.eventTime)
    def __eq__(self, other):
        return (self.eventTime == other.eventTime)
    def __ne__(self, other):
        return (self.eventTime != other.eventTime)
    
    def setTime(self, etime):
        self.eventTime = etime
    def setType(self, etype):
        self.eventType = etype     
    def setID(self, eID):
        self.eventTime = eID
    def setServiceTime(self, eServiceTime):
        self.eventTime = eServiceTime
    def setWaitingTime(self, eWaitingTime):
        self.eventTime = eWaitingTime
    def getTime(self):
        return self.eventTime
    def getType(self):
        return self.eventType
    def getID(self):
        return self.eventID
    def getServiceTime(self):
        return self.eventServiceTime
    def getWaitingTime(self):
        return self.eventWaitingTime

       
    
class Sim(object):
    #FutureEventList = Q.PriorityQueue()
    #Jobs1 = Q.Queue()
    #ServiceTimeQ1 = Q.Queue()
    #FutureEventList2 = Q.PriorityQueue()
    #Jobs2 = Q.Queue()
    #ServiceTimeQ2 = Q.Queue()
    #ToBeScheduledNextQ1 = Q.Queue()
    #ToBeScheduledNextQ2 = Q.Queue()
    
    def __init__(self, meanInterArrivalTime, k, p, alpha):
        print('Printing from initialization')
        self.meanInterArrivalTime = meanInterArrivalTime
        self.k = k
        self.p = p
        self.alpha = alpha
        self.FutureEventList = Q.PriorityQueue()
        self.Jobs1 = Q.Queue()
        self.ServiceTimeQ1 = Q.Queue()
        self.FutureEventList2 = Q.PriorityQueue()
        self.Jobs2 = Q.Queue()
        self.ServiceTimeQ2 = Q.Queue()
        self.ToBeScheduledNextQ1 = Q.Queue()
        self.ToBeScheduledNextQ2 = Q.Queue()
        self.queueLength1 = 0
        self.queueLength2 = 0
        self.numInService1 = 0
        self.numInService2 = 0
        self.lastEventTime1 = 0.0
        self.lastEventTime2 = 0.0
        self.totalBusy1 = 0.0
        self.totalBusy2 = 0.0
        self.sumResponseTime1 = 0.0
        self.sumResponseTime2 = 0.0
        self.sumWaitingTime1 = 0.0
        self.sumWaitingTime2 = 0.0
        self.numOfDepartures1 = 0
        self.numOfDepartures2 = 0
        self.sumResponseTime1To2Cumulative = 0.0
        self.sumWaitingTime1To2Cumulative = 0.0
        self.numDirect2 = 0
        self.num1To2 = 0
        self.eventIDCounter1 = 0
        self.eventIDCounter2 = 100000
        self.q1q2WaitingTime = {}
        self.q1q2ServiceTime = {}
        self.q2WaitingTime = {}
        self.q2ServiceTime = {}
        


    def initialization(self):
        timeToFirstArrival = self.generateRandomInterArrivalTime()
        arrival1ToBeScheduled = self.checkArrival1OrArrival2()
        if(arrival1ToBeScheduled == True):
            self.eventIDCounter1 = self.eventIDCounter1 + 1
            arrivalEvent = Event(timeToFirstArrival, 'arrival1', self.eventIDCounter1)
        else:
            self.eventIDCounter2 = self.eventIDCounter2 + 1
            arrivalEvent = Event(timeToFirstArrival, 'directArrival2', self.eventIDCounter2) 
        self.FutureEventList.put(arrivalEvent)
        return
    

    def generateRandomInterArrivalTime(self):
        random.seed(15)
        return np.random.exponential(scale=self.meanInterArrivalTime)
    
    def generateRandomServiceTime1(self):
        random.seed(15)
        x = generate_random_bp(self.k, self.p, self.alpha)
        return x   
    
    def generateRandomServiceTime2(self):
        random.seed(15)
        x = generate_random_bp(self.k, self.p, self.alpha)
        return x  
    
    def generateRandomServiceTime(self):
        random.seed(15)
        x = generate_random_bp(self.k, self.p, self.alpha)
        return x  
 
    def checkArrival1OrArrival2(self):
        x = np.random.uniform(0, 1, (1,1))
        if(x[0,0] < 0.25):
            return True
        else:
            return False
                
    def processArrival1(self, event, clock):
        self.Jobs1.put(event)
        self.ToBeScheduledNextQ1.put(event)
        self.queueLength1 = self.queueLength1+1
        
        
        if(self.numInService1 == 0):
            nextEvent = self.ToBeScheduledNextQ1.get()
            self.scheduleService1(clock, nextEvent.getID())
        else:
            self.totalBusy1 = self.totalBusy1 + clock - self.lastEventTime1

            
        #Schedule next arrival
        timeToNextArrival = self.generateRandomInterArrivalTime()
        arrival1ToBeScheduled = self.checkArrival1OrArrival2()
        
        if(arrival1ToBeScheduled == True):
            self.eventIDCounter1 = self.eventIDCounter1 + 1
            nextArrival = Event(clock+timeToNextArrival, 'arrival1', self.eventIDCounter1)
        else:
            self.eventIDCounter2 = self.eventIDCounter2 + 1
            nextArrival = Event(clock+timeToNextArrival, 'directArrival2', self.eventIDCounter2)            
        
        self.FutureEventList.put(nextArrival)
        self.lastEventTime1 = clock

        
    def scheduleService1(self, clock, eventID):

        serviceTime = -1.0
        while(serviceTime < 0):
            random.seed(15)
            serviceTime = self.generateRandomServiceTime1()
        nextDeparture = Event(clock+serviceTime, 'departure1', eventID)
        self.FutureEventList.put(nextDeparture)
        self.numInService1 = 1
        self.queueLength1 = self.queueLength1 - 1
        scheduledJob = Event(serviceTime, 'service', eventID)
        self.ServiceTimeQ1.put(scheduledJob)
        
    def processDeparture1(self, event, clock):
        finished1 = self.Jobs1.get()
        serviceDoneJob = self.ServiceTimeQ1.get()
        

        if(self.queueLength1 > 0):
            nextEvent = self.ToBeScheduledNextQ1.get()
            self.scheduleService1(clock, nextEvent.getID())
        else:
            self.numInService1 = 0
            
        responseTime = clock - finished1.getTime()
        self.sumResponseTime1 =  self.sumResponseTime1 + responseTime
        
        serviceTime = serviceDoneJob.getTime()
        waitingTime = responseTime - serviceTime
        self.sumWaitingTime1 = self.sumWaitingTime1 + waitingTime
        
        self.q1q2WaitingTime[event.getID()] = waitingTime
        self.q1q2ServiceTime[event.getID()] = serviceTime
    
        secondStageJob = Event(clock, 'arrival1ToArrival2', event.getID())
        self.FutureEventList.put(secondStageJob)       

        self.totalBusy1 = self.totalBusy1 + clock - self.lastEventTime1
        self.numOfDepartures1 = self.numOfDepartures1 + 1
        self.lastEventTime1 = clock

        
    def processArrival2(self, event, clock):
        self.Jobs2.put(event)
        self.ToBeScheduledNextQ2.put(event)
        self.queueLength2 = self.queueLength2+1
        

        if(self.numInService2 == 0):
            nextEvent = self.ToBeScheduledNextQ2.get()
            self.scheduleService2(clock, nextEvent.getID())
        else:
            self.totalBusy2 = self.totalBusy2 + clock - self.lastEventTime2

            
        #Schedule next arrival if this is a direct arrival
        eventType = event.getType()
        if(eventType == 'directArrival2'):
            direct = True
        else:
            direct = False
    
        if(direct == True):
            timeToNextArrival = self.generateRandomInterArrivalTime()
            arrival1ToBeScheduled = self.checkArrival1OrArrival2()
            if(arrival1ToBeScheduled == True):
                self.eventIDCounter1 = self.eventIDCounter1 + 1
                nextArrival = Event(clock+timeToNextArrival, 'arrival1', self.eventIDCounter1)
            else:
                self.eventIDCounter2 = self.eventIDCounter2 + 1
                nextArrival = Event(clock+timeToNextArrival, 'directArrival2', self.eventIDCounter2)    
            
            self.FutureEventList.put(nextArrival)
        self.lastEventTime2 = clock

    def scheduleService2(self, clock, eventID):

        serviceTime = -1.0
        while(serviceTime < 0):
            random.seed(15)
            serviceTime = self.generateRandomServiceTime2()
        nextDeparture = Event(clock+serviceTime, 'departure2', eventID)
        self.FutureEventList.put(nextDeparture)
        self.numInService2 = 1
        self.queueLength2 = self.queueLength2 - 1
        scheduledJob = Event(serviceTime, 'service', eventID)
        self.ServiceTimeQ2.put(scheduledJob)
    
        
    def processDeparture2(self, event, clock):

        finished2 = self.Jobs2.get()
        serviceDoneJob = self.ServiceTimeQ2.get()
       
        if(self.queueLength2 > 0):
            nextEvent = self.ToBeScheduledNextQ2.get()
            self.scheduleService2(clock, nextEvent.getID())
        else:
            self.numInService2 = 0
            
        responseTime = clock - finished2.getTime()
        self.sumResponseTime2 =  self.sumResponseTime2 + responseTime
        
        serviceTime = serviceDoneJob.getTime()
        waitingTime = responseTime - serviceTime
        self.sumWaitingTime2 = self.sumWaitingTime2 + waitingTime
        self.totalBusy2 = self.totalBusy2 + clock - self.lastEventTime2
        self.numOfDepartures2 = self.numOfDepartures2 + 1
        
        if(event.getID() > 100000):
            self.q2WaitingTime[event.getID()] = waitingTime
            self.q2ServiceTime[event.getID()] = serviceTime
            self.numDirect2 = self.numDirect2 + 1
        else:
            self.q1q2WaitingTime[event.getID()] = self.q1q2WaitingTime[event.getID()] + waitingTime
            self.q1q2ServiceTime[event.getID()] = self.q1q2ServiceTime[event.getID()] + serviceTime  
            self.num1To2 = self.num1To2 + 1
        
           
        self.lastEventTime2 = clock

    def generateMeanTimeStats(self):
        i = 0
        numQ1Q2 = 0
        sumQ1Q2WaitingTime = 0.0
        sumQ1Q2ServiceTime = 0.0
        for key, value in self.q1q2WaitingTime.items():
            sumQ1Q2WaitingTime = sumQ1Q2WaitingTime+value
            numQ1Q2 = numQ1Q2+1
        for key, value in self.q1q2ServiceTime.items():
            sumQ1Q2ServiceTime = sumQ1Q2ServiceTime+value
        return [self.sumWaitingTime1/self.num1To2, self.sumWaitingTime2/(self.num1To2+self.numDirect2), sumQ1Q2WaitingTime/numQ1Q2, self.sumResponseTime1/self.num1To2, self.sumResponseTime2/(self.num1To2+self.numDirect2), (sumQ1Q2ServiceTime+sumQ1Q2WaitingTime)/numQ1Q2, self.queueLength1, self.queueLength2]

            
        
    def reportGeneration(self):
        print('Avg Waiting time 1')
        print(self.sumWaitingTime1/self.num1To2)
        print('Avg Waiting time 2')
        print(self.sumWaitingTime2/(self.num1To2+self.numDirect2))
        print('Avg Response time 1')
        print(self.sumResponseTime1/self.num1To2)
        print('Avg Response time 2')
        print(self.sumResponseTime2/(self.num1To2+self.numDirect2))
        [avgQ1Q2WaitingTime, avgQ1Q2ServiceTime] = self.generateMeanTimeStats()
        print('Avg Waiting Time 1+2')
        print(avgQ1Q2WaitingTime)
        print('Avg Service Time 1+2')
        print(avgQ1Q2ServiceTime)
        #print('totalBusy')
        #print(self.totalBusy1 + self.totalBusy2)
        print('Wait queue in 1')
        print(self.queueLength1)
        print('Wait queue in 2')
        print(self.queueLength2)
        #print('QSize')
        #print(Sim.FutureEventList.qsize())
        print('Number of departures from 1')
        print(self.num1To2)
        print('Number of departures from 2')
        print(self.numDirect2)
        return
    
totalCustomers = 10000

avgWaitingTime1List = []
avgWaitingTime2List = []
avgWaitingTimeQ1Q2List = [] 
avgResponseTime1List= [] 
avgResponseTime2List = [] 
avgResponseTimeQ1Q2List = []
q1LengthList = [] 
q2LengthList = []

    
i = 0
while(i < 100):

    simClock = 0.0

    simu = Sim(4000, k, p, alpha)
    simu.initialization()
    while((simu.numOfDepartures2 < totalCustomers)):
        event = simu.FutureEventList.get()

        eventType = event.getType()
        eventTime = event.getTime()
        eventID = event.getID()
        simClock = eventTime

        if(eventType == 'arrival1'):
            simu.processArrival1(event, simClock)
        elif(eventType == 'directArrival2'):
            simu.processArrival2(event, simClock)
        elif(eventType == 'arrival1ToArrival2'):
            simu.processArrival2(event, simClock)
        elif(eventType == 'departure1'):
            simu.processDeparture1(event, simClock)
        else:
            simu.processDeparture2(event, simClock)
        del event
    [avgWaitingTime1, avgWaitingTime2, avgWaitingTimeQ1Q2, avgResponseTime1, avgResponseTime2, avgResponseTimeQ1Q2, q1Length, q2Length] = simu.generateMeanTimeStats()
    print([avgWaitingTime1, avgWaitingTime2, avgWaitingTimeQ1Q2, avgResponseTime1, avgResponseTime2, avgResponseTimeQ1Q2, q1Length, q2Length])  

    i = i+1
    avgWaitingTime1List.append(avgWaitingTime1)
    avgWaitingTime2List.append(avgWaitingTime2)
    avgWaitingTimeQ1Q2List.append(avgWaitingTimeQ1Q2)
    avgResponseTime1List.append(avgResponseTime1)
    avgResponseTime2List.append(avgResponseTime2)
    avgResponseTimeQ1Q2List.append(avgResponseTimeQ1Q2)
    q1LengthList.append(q1Length)
    q2LengthList.append(q2Length)


In [2]:
import numpy as np
#Post proccessing
#Bounded Pareto
#p = 0.75
#Mean [219648.49922525964, 177906.49236970182, 225555.18451469034, 180950.68852944893, 3.3599999999999999, 234.00999999999999]
#Std dev [395399.53407107829, 480545.21681747248, 395506.08024964511, 481177.64595333411, 13.58493283016151, 1843.8958999629024]
#Bounded Pareto
#p = 0.5
#Bounded Pareto
#p = 0.25

meanListbpAll = []
stdListbpAll = []
meanListexpAll = []
stdListexpAll = []
meanListbp75 = [219648.49922525964, 177906.49236970182, 225555.18451469034, 180950.68852944893, 3.3599999999999999, 234.00999999999999]
stdListbp75 = [395399.53407107829, 480545.21681747248, 395506.08024964511, 481177.64595333411, 13.58493283016151, 1843.8958999629024]
meanListbp5 = [323858.02329539211, 148244.73582794555, 329910.92965205526, 151215.69256493339, 48.0, 60.18]
stdListbp5 = [1298345.1613539322, 518117.08069191774, 1299116.8208522268, 518267.84107370942, 323.88553533617397, 248.6126054728521]
meanListbp25 = [86402.900471894187, 78437.065326520969, 92290.154471888542, 81389.79942494593, 0.35999999999999999, 29.300000000000001]
stdListbp25 = [86979.593895671846, 85097.918206125541, 87132.497920973488, 85202.45106585219, 2.2913751329714649, 88.128485746664225]
meanListexp25 = [9788.9020727707721, 9103.6113216199265, 15794.204128363968, 12108.421947341689, 0.089999999999999997, 1.8899999999999999]
stdListexp25 = [804.43862350625716, 772.0450547592576, 815.1974377292197, 786.07390558090447, 0.37669616403674738, 2.6263853487255067]
meanListexp5 = [10914.531119210271, 9109.9339069829875, 16921.35086262635, 12116.674705749483, 0.23000000000000001, 2.46]
stdListexp5 = [911.62812432734461, 847.83962695225273, 929.88870784433857, 862.503609284096, 0.56311632901204345, 3.0444047037146689]
meanListexp75 = [12815.791463608832, 8979.9783873408451, 18813.889520103163, 11980.716854986553, 0.56999999999999995, 2.1200000000000001]
stdListexp75 = [924.2364684268415, 837.51775553247512, 953.2591842356245, 856.63296750569464, 1.3285706605220515, 3.5079338648269869]
meanListbpAll.append(meanListbp25)
stdListbpAll.append(stdListbp25)
meanListbpAll.append(meanListbp5)
stdListbpAll.append(stdListbp5)
meanListbpAll.append(meanListbp75)
stdListbpAll.append(stdListbp75)
meanListexpAll.append(meanListexp25)
stdListexpAll.append(stdListexp25)
meanListexpAll.append(meanListexp5)
stdListexpAll.append(stdListexp5)
meanListexpAll.append(meanListexp75)
stdListexpAll.append(stdListexp75)

avgWaitingTime1List = [3853.3547542414485, 4315.2965035913185, 3970.4795252935746, 3617.6412912050187, 3591.6476402043254, 3651.82209183666, 3700.630319693443, 3984.0015086960857, 3945.5858648242443, 3953.911778215751, 3604.995436118869, 3782.0223849887507, 3798.9304805596616, 3800.090570002604, 4419.788946825326, 3631.9718794729465, 4118.093082552395, 3745.3458673540736, 3988.7144538451194, 3811.7146856364225, 4316.973063156641, 3688.92570717118, 3666.6914951589124, 3675.9699497190086, 3706.1120090205145, 3720.8682149030487, 3669.386970982413, 3483.6963804880425, 4391.735021857566, 3602.937333396489, 4137.722117697472, 3854.3797351757407, 3834.5467567119895, 4026.466967013045, 3873.932320447224, 3975.607013647905, 3864.5950690852396, 3813.2120504354043, 3468.0163420188856, 3610.250123124762, 3692.802502721558, 4055.829363324555, 3679.4275653665086, 3409.3164739477193, 4064.037886213325, 3600.2189943689064, 4257.512164414425, 4013.0090088655616, 3913.7155013307997, 3641.8462008282286, 3258.3134788235816, 4217.509788769166, 4084.705389065791, 4054.7833365390047, 4240.505545928702, 3586.8569909873536, 4196.459958509791, 3978.894159482025, 3304.835869702973, 3706.229755520423, 3841.0670380597594, 4172.112876015079, 3716.483911050666, 3870.5826276071216, 4096.3839604383165, 3710.736444912902, 4097.9964538851045, 3701.3260755925185, 3818.7279274223815, 4066.6189859078036, 3675.275212347748, 3864.056978927443, 4120.221491622833, 3500.5139067945684, 3930.5972475470608, 3861.8084614794207, 3944.5225393593623, 3812.290774441836, 3435.410939597341, 3927.283049546275, 3550.690168603057, 3824.1412602460487, 3835.8605816482104, 3414.7035904938994, 3696.4221151030592, 4050.5148319224136, 4048.7644766057597, 3788.7603912729255, 3717.159901585454, 3837.531499820742, 4010.0911917617623, 3777.222284437341, 3771.4233607423553, 3885.1178768167897, 4024.9095506397985, 4186.149038784598, 3467.2864867721905, 3855.511112111796, 4283.232594234595, 3626.5951254932916]
avgWaitingTime2List = [8644.819359906725, 8743.880891392455, 10255.734711567136, 8630.517224394758, 8903.771599314396, 9676.065325770762, 8669.291264139925, 8457.132423760744, 8539.599858693411, 8321.628317157845, 7528.92845501933, 9565.535104420542, 8508.309459521719, 8151.186294468901, 10186.867854120874, 9787.112620506325, 7694.260559866616, 7891.242829453154, 9049.633312786242, 8818.268302355964, 8880.0223230075, 8087.8728183949, 9329.582973186056, 10676.503359110697, 8978.486903421965, 10089.173799465456, 8322.444017025342, 7699.955563152911, 10086.063217857387, 8043.454993950966, 9115.067327272103, 10921.716636440045, 9085.892286521886, 9687.494595828713, 9064.648214467148, 10059.956994052396, 8904.161963633298, 9174.511383627892, 7727.988098124016, 9957.327553050827, 9448.159766735815, 8684.811887597101, 8295.711821088435, 8111.477567041968, 9010.524164468474, 8632.59773822053, 10019.38398848557, 9541.046994573593, 7827.608411908309, 9977.508822194302, 8335.03110154854, 10984.388891632812, 8560.149118688856, 9213.960618464462, 7860.500663422244, 8158.5133617343945, 10450.58273697251, 8524.921612443579, 9367.712792423861, 10213.842708291873, 8936.177856097702, 9446.215857526864, 8457.76785412458, 9217.11730417467, 8611.865312583894, 8885.151507642928, 7957.993746783295, 9637.235312724606, 8760.561483425754, 9254.793765729934, 9086.719381351006, 8340.749850997649, 7933.48404607286, 8644.916592297706, 8603.946077033283, 8433.338237536589, 9300.041245073015, 8157.081455174526, 8196.957813262996, 8854.04304241646, 9337.171996629528, 8506.076235757228, 8485.952516589066, 8689.789705702759, 9139.648333676914, 10187.956016784918, 9762.408391216766, 8427.12673028893, 9539.960125393189, 7925.165206966511, 7765.1852564889905, 10495.811834736036, 10948.967821220238, 10460.471070252497, 9433.012873466909, 9168.580920798751, 8085.200511105293, 7787.6779928340775, 10133.345796259522, 7867.626097739515]
avgWaitingTimeQ1Q2List = [12417.595060209494, 13007.124236802476, 14154.556131497282, 12169.759278756635, 12449.297857595046, 13279.905169190462, 12273.077337168545, 12294.058692671559, 12380.985811167235, 12301.257966991352, 11097.253528243515, 13357.369773850498, 12405.824945654127, 11914.27272401695, 14733.157351472326, 13252.450023598198, 11803.365151093285, 11562.126979754832, 13178.978181495897, 12628.626845909874, 13144.084023852398, 11726.798342891041, 13036.493155802995, 14332.97469470071, 12696.527396757438, 13720.377853176467, 11989.426956850763, 11180.914058530961, 14568.46638652506, 11696.381741396583, 13154.18803845578, 14666.483964450385, 12821.608580506827, 13625.128686591748, 12937.664805897892, 14107.461682743518, 12839.285926729382, 12980.6015243705, 11249.997448584909, 13651.52384316335, 13226.789349277664, 12705.144021681961, 11913.379578156935, 11412.369376240445, 13027.083379041545, 12210.830284826272, 14188.541131602975, 13644.046531148299, 11743.769654320458, 13603.836023683409, 11545.361818126747, 15092.836387146928, 12585.833223412614, 13272.840400680763, 12194.358916516067, 11748.46430669007, 14619.797588938092, 12555.871335008627, 12722.671122788657, 13888.607985397568, 12744.418557297304, 13589.070560668159, 12049.101968422387, 12962.005204144716, 12638.86454515942, 12631.254098723568, 12052.551905541026, 13307.231874757925, 12607.995680555783, 13405.336962812296, 12829.068167304951, 12205.48093599361, 12104.96056484073, 12071.146782157522, 12537.945151426613, 12277.908560914026, 13271.55169702063, 11977.822094800062, 11579.748330215436, 12808.989516268712, 12920.244565777908, 12385.007556096241, 12399.214265837825, 12177.401435331032, 12919.350223699303, 14151.344176517232, 13825.335310654678, 12257.097444276458, 13288.642173194537, 11846.66062329551, 11835.650830868139, 14415.351940076645, 14707.636188422288, 14262.525911018083, 13334.772546567752, 13468.279553958848, 11571.155238365458, 11628.708779163018, 14358.011732728803, 11484.438132204314]
avgResponseTime1List = [6877.939647663233, 7369.009611552463, 6959.28195101842, 6597.223124492101, 6577.54140836397, 6605.155992126646, 6696.355770481527, 7013.003078036065, 6967.472189556071, 6956.53364367819, 6599.5978266673665, 6792.341418505489, 6820.883103528884, 6816.882267039174, 7429.120288173036, 6576.885714485399, 7187.840345304919, 6725.152351130856, 7040.794080248183, 6834.94030245739, 7339.984087375409, 6698.251401147097, 6628.958579266698, 6666.712145068265, 6679.547151194916, 6692.856269385091, 6627.861664540286, 6417.067582398226, 7476.269924031849, 6599.7313652329085, 7128.953111196551, 6876.297755928557, 6815.224781580602, 7006.275798382567, 6863.338655187236, 6958.771992437633, 6862.922791754576, 6830.326699455116, 6429.734204891386, 6544.605536080817, 6653.110887379674, 7068.5206978407505, 6644.266165425878, 6384.966954619331, 7142.769982109893, 6582.606459030731, 7347.06841296204, 6981.891502373639, 6930.87376666276, 6587.032870260525, 6209.225720827227, 7255.831658885915, 7092.844294605292, 7086.97341063581, 7215.708830371851, 6537.906459351499, 7194.66453766631, 6965.801169936793, 6317.358924615308, 6722.392513600757, 6866.925248646645, 7207.296166801452, 6686.577827856466, 6813.676507134888, 7116.713283986571, 6698.001801703739, 7123.458564723588, 6674.166027254591, 6839.847607749608, 7123.238025792693, 6654.557730098256, 6850.456241127203, 7153.6216713875865, 6513.548567368229, 6910.378599700515, 6846.541408986198, 6974.629560391373, 6793.737793232345, 6391.43023814702, 6874.512539369413, 6556.877305477817, 6810.261481240483, 6837.6900594714725, 6421.448097103796, 6729.275954138639, 7046.4483049550745, 7052.818254869782, 6778.676080526098, 6712.333930746783, 6828.030531775258, 7025.991696553451, 6740.181306899127, 6773.509200392485, 6888.444177926634, 7042.852919411386, 7213.0881786010505, 6427.483644028802, 6850.607793446644, 7300.680158757161, 6632.226039577536]
avgResponseTime2List = [11618.154718053745, 11707.896621867696, 13298.244796472805, 11623.028366401724, 11916.397589984364, 12693.321239807741, 11648.853341463699, 11438.89031208372, 11529.86923215172, 11332.616248013523, 10522.91854520292, 12571.529303340247, 11486.073175042793, 11099.010726251627, 13180.049658903688, 12813.747977763218, 10626.987842655846, 10843.9836468021, 12100.336040896249, 11760.313850482511, 11861.368013967971, 11048.848917935451, 12292.417064150426, 13704.177956078365, 11986.174542874845, 13117.986299851715, 11312.447787809524, 10647.112834962105, 13130.943067919674, 11043.719346445254, 12092.176563890269, 13979.727769261614, 12063.538891152506, 12687.31048499199, 12056.472384968394, 13112.119356972607, 11874.172911581623, 12178.379294053939, 10711.9066270485, 12953.632456718964, 12522.12382150457, 11652.694947121883, 11269.849975170931, 11087.184628174371, 11975.66318610185, 11634.899768190739, 13067.459791228657, 12575.977542455124, 10826.434271713526, 12969.280795387, 11307.971773989017, 14034.203848757998, 11554.960474125168, 12196.846794646817, 10874.538347412037, 11120.592108032559, 13436.399211795482, 11497.176036995215, 12417.217157020172, 13240.224995545595, 11953.76964877093, 12454.308429457797, 11455.965887824676, 12207.810478989884, 11636.324760981726, 11906.385322524317, 10963.070058517347, 12649.217526135006, 11746.55389804351, 12274.933398836734, 12094.315780749994, 11333.879755471888, 10918.109756227903, 11647.097320248095, 11576.60981907787, 11411.620967208659, 12352.091395235164, 11123.630765700078, 11238.828749518721, 11899.973934926078, 12354.465218855217, 11519.277102403092, 11477.611329445414, 11714.282167769987, 12101.290119945821, 13232.360182438651, 12811.936848078105, 11396.586870939977, 12597.958404128363, 10925.53616353736, 10718.335674067384, 13549.352977313, 13991.760515135897, 13461.463849015647, 12427.8341124198, 12186.799860556692, 11081.805857637428, 10727.271773162141, 13158.8190434168, 10863.884492295898]
avgResponseTimeQ1Q2List = [18412.668438366, 19018.900392131116, 20189.089652358543, 18117.26072317604, 18441.89591840607, 19256.42722155234, 18255.938761974547, 18295.053449271607, 18384.02737484104, 18343.056200267703, 17061.13937906399, 19363.22462959873, 18405.2376763133, 17884.291811846244, 20756.429240891837, 19231.45132514256, 17810.7220873395, 17491.926658998473, 19279.421791081066, 18614.113763699075, 19144.54892672641, 17706.43311666147, 18958.09374471354, 20364.224821430922, 18667.542102782132, 19732.05431556572, 17959.583397487742, 17065.221068374052, 20714.657042800787, 17668.34211804511, 19123.40951109126, 20758.027976827583, 18779.01188720191, 19607.183392112296, 18935.098764610615, 20143.077523135325, 18810.294554629687, 18985.918407953355, 17153.961502961785, 19601.033231484584, 19260.79066130603, 18652.789865073762, 17862.73853612934, 17357.686268866804, 19053.694457535996, 18209.70075905082, 20311.633319176595, 19639.959329711743, 17752.980634340453, 19535.669147561577, 17470.36371362508, 21179.732481908988, 18588.194508700817, 19296.837221170787, 18192.73436251299, 17702.47275022507, 20591.787747947357, 18515.863740193538, 18793.862959923128, 19924.909504941912, 18800.848621240726, 19604.788257742824, 17988.384315368174, 18879.50396749145, 18671.97763093584, 18613.82166642925, 18067.339399732235, 19287.1620164929, 18612.55213675692, 19452.284935888987, 18800.251512559407, 18210.605013329307, 18136.137984010587, 18074.242291128303, 18500.516661024558, 18211.691746429824, 19336.8356552376, 17918.281779999998, 17575.600388210652, 18789.280538270013, 18944.947660383245, 18381.18044938082, 18401.544824814842, 18204.697224980755, 18875.861800710423, 20210.93445099107, 19871.21210788826, 18216.9239213776, 19359.997487808298, 17834.526743699615, 17778.35543546413, 20441.912955472024, 20758.803502475785, 20270.37538988237, 19358.863895029364, 19545.797755539836, 17516.83182605767, 17572.74377698965, 20436.100637189036, 17490.837773082905]
q1LengthList = [0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 2, 3, 0, 0, 1, 0, 0, 1, 3, 0, 0, 1, 0, 3, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 8, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 7, 0, 0, 0, 0, 0]
q2LengthList = [0, 1, 0, 0, 0, 0, 1, 1, 17, 3, 3, 4, 3, 5, 1, 4, 5, 0, 1, 0, 0, 0, 5, 0, 2, 1, 0, 0, 2, 3, 1, 6, 0, 0, 1, 0, 0, 2, 7, 0, 1, 0, 4, 0, 1, 0, 23, 3, 0, 2, 3, 11, 2, 0, 1, 2, 0, 3, 5, 0, 0, 14, 0, 4, 0, 3, 0, 0, 3, 2, 3, 0, 0, 0, 4, 2, 4, 0, 2, 0, 0, 0, 2, 1, 2, 0, 0, 2, 0, 3, 3, 0, 5, 1, 1, 1, 0, 8, 0, 2]



meanList = []
stdDevList = []
listOfLists = [avgWaitingTimeQ1Q2List, avgWaitingTime2List, avgResponseTimeQ1Q2List, avgResponseTime2List, q1LengthList, q2LengthList]
for elementList in listOfLists:
    array = np.array(elementList)
    meanList.append(np.mean(array))
    stdDevList.append(np.std(array))
print(meanList)
print(stdDevList)

[12815.791463608832, 8979.9783873408451, 18813.889520103163, 11980.716854986553, 0.56999999999999995, 2.1200000000000001]
[924.2364684268415, 837.51775553247512, 953.2591842356245, 856.63296750569464, 1.3285706605220515, 3.5079338648269869]


In [ ]:
from matplotlib import pyplot as plt
#Exponential
#p = 0.25

avgWaitingTime1ListExp = [718.1581652532781, 761.556457881731, 697.6775542781846, 667.86457226752, 650.3266919152505, 673.6310133598134, 709.2041646092728, 763.3474447258233, 889.7309918058037, 647.2864354369459, 724.9666546735804, 676.4559270452372, 618.7117051107516, 739.5276419665399, 721.6557408614783, 874.0690536181181, 771.0109649266763, 805.3487221119427, 571.7089464998774, 717.7175268120126, 769.4514426964104, 689.6235578018932, 722.8781848335236, 734.5284133185431, 798.1624512487891, 649.3014580240724, 742.7798223194283, 741.3335750507653, 731.9988845468984, 682.4465260255129, 634.6796840469724, 658.1287147716587, 671.7870439345695, 588.5691103208878, 711.3259179553959, 750.0296902674954, 663.3910681169054, 708.0581128962365, 850.7733946979878, 756.3880654386936, 641.7215607369484, 654.3690445398883, 833.1517782477065, 684.3868217999695, 689.0349781275122, 723.8784199511497, 860.0003051523116, 750.9645004185544, 755.5361958518318, 643.9513435357253, 702.2418582261485, 611.7723066035039, 747.1525259074011, 719.8542038628703, 674.686873520219, 724.6545445086593, 674.0406158435048, 564.0219802343598, 614.2824739396492, 584.1159678413215, 727.573728208606, 715.6472683919487, 656.0510940888478, 733.9972411978029, 698.2414753756917, 763.5228573047355, 673.297830312629, 675.465954421509, 784.5841815372672, 821.8110975751449, 761.313423145669, 752.8826484682331, 751.6010820357409, 626.5846273233527, 617.3498403827166, 824.265466807563, 664.5321308380563, 656.183114402013, 703.2897608625987, 634.3220054870977, 705.773375260806, 604.351924875967, 717.1254548614377, 722.0318529815105, 892.2590669274639, 709.0732276166949, 576.672149009788, 585.1292655784122, 699.7963854030771, 784.1961406729448, 627.1033644315969, 782.980942247372, 707.7155790238355, 658.8346634996664, 691.5290565192678, 688.4418150564464, 651.5114797952473, 665.3484188316017, 666.6069655102691, 715.5464372914336]
avgWaitingTime2ListExp = [9941.960957147168, 9239.22663664167, 9444.88206544476, 8361.982327281732, 8515.516207933902, 9309.143462649054, 9536.571857332614, 8944.262650108003, 9875.692749908012, 10066.385868922049, 8828.611739844302, 8395.657243168776, 8821.35455005128, 9654.477720991146, 9471.813456763755, 9452.527297444976, 9420.366417508792, 9949.138074107503, 9073.3177098348, 8897.515210251775, 8259.022610159725, 9374.12971582884, 9235.950674234873, 8321.33735698967, 8971.376646256176, 8881.66685615674, 9626.747940576119, 10497.531695698723, 10335.331647316101, 8197.365553737465, 8369.330611335157, 10165.704956142768, 9124.302937241213, 8313.945826003597, 8493.571356948838, 8741.907042547084, 9264.454634718688, 8934.965697555996, 9717.380837941648, 9757.73598311906, 9659.383360589714, 9357.86625926904, 9654.189922574804, 8900.184200607982, 8010.497112117383, 9897.919456880094, 11754.379975825708, 9367.312231040136, 10828.23706489645, 9175.04966259847, 8165.306846978701, 9485.731348980085, 9626.552432211545, 7699.301911806288, 8521.26716142256, 8925.891370957854, 8338.325014897158, 8964.71456858969, 10274.374739274543, 7881.98166646692, 8375.349746350197, 7912.457615328036, 7763.605187450397, 8490.00999320426, 9981.911594352003, 8350.620261801128, 9206.263540303527, 8744.242857702713, 8429.003821007294, 10894.538617269702, 8270.506209588335, 8395.115965284627, 9095.69965859995, 10847.812823389713, 7593.875717991862, 7582.582795105802, 9665.386178784085, 8160.215746318721, 9487.329756004137, 9217.972983724165, 9176.211547312172, 8894.235376012679, 8874.602014836039, 8323.708866134586, 8967.502130720002, 8832.36525140263, 9618.10984881025, 9230.92899388442, 9982.36663251275, 8501.146171248285, 9299.518762055632, 9760.880095568671, 9140.137352469257, 9293.120895943617, 7779.16545686115, 9114.837306660631, 8556.023710748408, 9309.733465235784, 9372.917341879338, 9596.48878030611]
avgWaitingTimeQ1Q2ListExp = [10660.994542316375, 10070.288738856447, 10562.42553267714, 9217.33282866893, 8860.807136827365, 9978.70705111745, 10275.542506660122, 9251.725437189953, 10458.209375611625, 10729.18662517005, 9523.988462103984, 8941.300610732242, 9527.384403269523, 10522.493639143235, 10381.283564189596, 10176.268123777028, 10361.567537622586, 10901.893830446912, 9336.421846530715, 9780.974027704384, 9110.501259324277, 10071.325087335985, 10142.53985986205, 8913.869108724379, 9932.219229013903, 9205.694677801941, 10434.416976859116, 11357.423333406012, 11104.273959776, 8703.87279361513, 8833.82798806739, 10481.005896308003, 9779.580496756262, 8987.853608135978, 9321.730621976392, 9398.545821015332, 9636.66828707302, 9618.935038720225, 10430.259850699114, 10401.798918060229, 9986.697655284106, 10020.964099001207, 10272.800851434307, 9507.407688619547, 8869.62888441363, 10579.903882045071, 12514.797213340547, 9800.116236412741, 11842.82724864815, 9804.05834172105, 8840.301972177247, 9767.960131297694, 10463.35338968239, 8480.33368721015, 9627.107521110387, 9714.957688721344, 9477.110109419633, 9424.9510548471, 10615.228813240337, 8396.479649612806, 9093.752711679974, 8327.486507574462, 8179.583480684606, 9306.26467382854, 10585.562312565311, 8919.280793913791, 9732.214517856066, 9505.924899809248, 9282.003560711597, 11635.416954521676, 8934.845261021717, 9186.409091495341, 9608.176859455414, 11432.944752809322, 7931.09000441811, 8452.580252548441, 10203.038476114825, 8993.366839567447, 10110.426713851592, 9674.435830392458, 10367.242244612746, 9239.70398852464, 9519.420735745241, 9374.717983833041, 9672.007539760038, 9439.550249812717, 10223.44200364679, 10129.212244694076, 10703.986522609057, 9170.069037540572, 10125.038416652176, 10401.842810555829, 10015.680033684435, 10219.772657866453, 8445.984639817374, 9680.998683928776, 9321.688775948229, 10119.477658467575, 9981.668163528828, 10251.773639642926]
avgResponseTime1ListExp = [3746.2772950442004, 3825.6224300881704, 3688.3010995064546, 3562.9901154027793, 3597.594779998061, 3708.525388328163, 3614.671532622819, 3802.622549502467, 3995.8785470836656, 3620.6767675052783, 3718.712279818132, 3706.0497825905586, 3503.0005692451973, 3741.195951505326, 3683.884785332444, 3973.016008347719, 3820.703303033584, 3859.720886081893, 3580.573945023885, 3708.8267094780563, 3830.6663373300657, 3741.788299338453, 3686.0233167519373, 3738.297600806178, 3784.35309765908, 3615.8111203710882, 3823.6033495740608, 3786.1257487764724, 3682.620145759416, 3692.3398627993497, 3624.563679272998, 3627.349959582118, 3638.1984220928157, 3525.538939596343, 3681.2089316909246, 3694.969269100097, 3750.510988798155, 3763.6824355580375, 3898.021998157445, 3769.6679102704807, 3547.6981669150305, 3600.7080599157525, 3914.0469911847977, 3687.919329452362, 3654.1299343354053, 3788.682428834, 3943.8659116347685, 3717.948293345188, 3636.6729494623405, 3542.1534438626245, 3707.5903839614466, 3719.8759699976345, 3762.1692868449027, 3744.666878139218, 3745.4755073484102, 3707.4640821598286, 3646.568926749402, 3440.587899167385, 3572.5473269809704, 3608.674675860611, 3724.5699549883184, 3782.107626259777, 3584.8090754041864, 3668.739668248638, 3735.5983386734065, 3800.318996097182, 3693.1704036421047, 3633.2773841745097, 3810.330991054529, 3911.40247843132, 3781.3284085113237, 3827.8515159335193, 3711.311977605997, 3590.2361672481015, 3590.060966912211, 3771.5323830814805, 3669.1335757148845, 3677.4397408818163, 3746.7955441638883, 3673.3123465074295, 3675.2563135768723, 3535.905213781705, 3642.6309788955377, 3624.4696630452586, 3932.7119026876667, 3740.126965278816, 3533.9468617926386, 3552.753909145467, 3685.658248127099, 3800.632393271986, 3675.3925081212105, 3868.9277665383074, 3726.4338558613454, 3564.3304564018945, 3739.5812968538544, 3704.958797665097, 3608.5136838054605, 3722.151634647691, 3706.579346584907, 3725.363772646757]
avgResponseTime2ListExp = [12947.119513014271, 12264.541127005252, 12413.431678347257, 11381.606742546262, 11497.143513344341, 12328.958236908102, 12547.7005919698, 11918.160526111924, 12909.82151367639, 13083.073277968308, 11834.803328346894, 11368.875988206104, 11856.67722087354, 12688.518503860098, 12488.280983839331, 12428.872116078961, 12430.786643918822, 12978.027809341582, 12052.937924647395, 11934.273576910224, 11245.62066290393, 12395.361801254126, 12267.921140811372, 11299.646052239294, 11955.055727538946, 11886.611533424704, 12677.178357111556, 13516.683978663856, 13349.264289281145, 11177.3492959072, 11392.72784299354, 13187.312009714002, 12147.151415818762, 11362.77589041398, 11481.690286518895, 11765.535928493506, 12259.826500000992, 11912.609399926401, 12737.647401126665, 12771.152156427763, 12701.686730467365, 12377.080324138038, 12634.132843519426, 11885.061729942605, 10985.195251299938, 12943.099654225052, 14807.36183672974, 12376.494182799626, 13825.99741021427, 12141.057460557267, 11144.879751337367, 12493.312315361554, 12669.995153776328, 10687.913870968003, 11486.097712390756, 11911.42851707778, 11307.195180047845, 11963.057060672107, 13314.107515303562, 10876.888463591806, 11353.154101415475, 10868.721753714948, 10761.218518842386, 11474.596932146513, 12977.457225347805, 11375.176430084963, 12250.965585510598, 11773.397791526202, 11425.188331244382, 13921.82396098534, 11256.38183195193, 11377.238525118486, 12095.834379457674, 13839.364188598636, 10565.87619455041, 10518.770906957094, 12691.627686170292, 11179.121575463098, 12496.262368191743, 12251.12944549135, 12162.185652198177, 11931.878210392415, 11825.72748043066, 11289.4653911809, 11961.553522030974, 11823.95510524186, 12632.873249754906, 12222.50830855811, 13031.886019174772, 11474.263140161362, 12304.928515014833, 12818.228378814507, 12146.95944070135, 12335.350517303774, 10769.413388457406, 12089.234338825554, 11533.30829641956, 12312.855861822816, 12383.746776559954, 12663.7899564499]
avgResponseTimeQ1Q2ListExp = [16649.15735743697, 16132.671234304287, 16534.028996475918, 15115.384658795101, 14803.546770442097, 16018.821734797915, 16215.522241702818, 15259.472753356033, 16568.175884312826, 16739.72129474024, 15548.359024734069, 14964.260506295877, 15530.172493697724, 16559.019411483176, 16391.383657243518, 16305.525740240757, 16381.874203421668, 16941.400990871058, 15345.190155866816, 15717.065037705455, 15138.288283726037, 16195.401349110587, 16174.857481922516, 14961.662704335908, 15896.720652755394, 15239.52944400621, 16555.075969735302, 17418.84043970481, 17198.033546238363, 14739.863182270788, 14783.399945058307, 16479.47936160353, 15737.807137160178, 14977.340115899568, 15251.079658336857, 15387.377027508837, 15665.127680101865, 15555.18441244124, 16490.47060234264, 16358.157248533276, 15960.41021222957, 15993.463410267104, 16397.46206719221, 15396.266681795007, 14832.110282628537, 16758.203419007437, 18623.649566379103, 15831.731723446714, 17696.184850368878, 15700.86105645809, 14880.058546814396, 15841.277516693566, 16472.15609341724, 14593.980637971335, 15628.578077993967, 15653.813118505775, 15427.207733679119, 15342.732227856874, 16630.259997720183, 14416.475427028074, 15037.189402241624, 14361.007340114293, 14202.252611996175, 15261.589450042373, 16512.868768269294, 15005.51722180813, 15838.7909568321, 15404.844048677078, 15289.897877728392, 17697.142247457923, 15015.66844627937, 15275.254954977847, 15622.626043766317, 17350.59051519355, 13924.07336311052, 14365.970369444816, 16272.955564903059, 15066.72417461306, 16091.805288692014, 15842.83995871811, 16204.41918153967, 15201.694293420045, 15344.840260167568, 15255.645540103169, 15629.931814878706, 15468.90659354524, 16170.19992778197, 16033.318448909231, 16645.332836158428, 15129.072371466547, 16184.229564844445, 16513.283137578113, 16012.50230533244, 16242.433689239899, 14494.106120572489, 15576.790792843547, 15266.259554584101, 16260.827907284372, 15947.541144098044, 16398.135709012993]
q1LengthListExp = [1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
q2LengthListExp = [0, 3, 1, 0, 1, 0, 0, 4, 2, 0, 5, 0, 0, 0, 0, 1, 9, 1, 3, 5, 2, 0, 2, 0, 3, 0, 0, 0, 3, 2, 0, 0, 3, 0, 2, 3, 10, 0, 0, 10, 7, 1, 2, 1, 4, 2, 2, 0, 1, 0, 8, 1, 1, 0, 8, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 10, 8, 1, 0, 6, 0, 2, 0, 0, 4, 4, 8, 6, 0, 1, 0, 2, 1, 0, 0, 3, 0, 2, 0, 0, 3, 3, 0, 0, 0, 0]

#Bounded Pareto
#p = 0.25

avgWaitingTime1ListBP = [1081.001621707308, 32072.359013112942, 2870.5760666610308, 3997.3125099596182, 1797.7012520987803, 812.97837766971168, 1701.9638969339132, 4544.061371221519, 1414.3774132637886, 2060.6511049865117, 3934.4737684923193, 7732.8699502987993, 1853.4735659171133, 14473.737210788349, 4494.4382828812049, 68208.229521545494, 1139.0898222880576, 7550.1392672698503, 2242.2049141438474, 762.50530692399559, 8577.1280113593584, 1081.4476939855665, 2329.9421675350732, 1879.573565713604, 1694.1056479083836, 1253.4312213696564, 7145.879143907614, 8439.6100033107341, 2749.7782463476547, 736.50672835122236, 2979.9388545162155, 2322.2354268667486, 1457.492870581453, 1276.8954863243075, 7189.4006801702471, 9454.9779234406051, 1967.4594737718646, 7407.9881225680483, 12342.238910292521, 11267.293272670377, 9469.2657013701028, 3792.8542394724955, 1839.3312507587341, 2190.5531107679494, 8964.2460961074103, 10317.052550591965, 1294.5044841548979, 152687.59080202313, 2856.9521144459018, 1243.4727403946474, 2490.0279311816466, 1602.2178762220528, 1198.4819475934953, 8757.2069978703039, 14458.936937624425, 1018.2243537766876, 3592.8403180352261, 6710.8977290839475, 1255.2155328350877, 19894.102960654556, 5030.8386125466923, 2436.9613004329776, 2224.0417573554596, 3304.385358086533, 2660.6296897868651, 1152.0979815060241, 916.61874468892074, 6921.4470786883721, 1226.4129855378421, 10227.706507606368, 1122.194473371271, 2166.5788302295996, 1984.8974861868994, 11379.40724766104, 7950.1517701151188, 2803.6511870459631, 6524.675299342337, 995.83839102334116, 2190.8351191527545, 1771.2936389901274, 12597.143942295066, 3497.4774398747209, 109788.26123655947, 1699.7959022844109, 2664.8363677200236, 2708.2167630326699, 1636.9570800097945, 2246.472907527736, 2812.3301588065483, 2135.1404581551296, 3463.4533018922739, 11412.102560743117, 1894.3255841149123, 2552.6442167484902, 3008.2770727843272, 1140.2107070418353, 2686.9776697576949, 7767.1545678957773, 1306.3227243127346, 2927.9605203484057]
avgWaitingTime2ListBP = [52281.629591361991, 351937.63711987727, 76500.6575733973, 116153.91883232408, 50066.955627588024, 58942.440290146813, 65122.73747900311, 19440.80025511589, 78633.324569414559, 55403.609352190921, 99148.519211298728, 42366.05383310013, 71551.188489897933, 46477.061678654551, 169740.496188474, 21280.847824201628, 55552.109426360381, 23773.400601328634, 85566.698995551938, 20148.830649432552, 20592.050227325977, 35007.665914558427, 24100.659943684819, 92593.813566023309, 24375.412906683312, 65535.27499464812, 16196.007704254916, 29698.125711507098, 505096.54232416925, 78654.077950885141, 30595.918843121726, 35246.75482778385, 67276.652400820487, 223530.68955333513, 93370.323284853541, 152214.98107276735, 47456.043727950862, 52143.727005702356, 38689.784636044205, 31422.64974198649, 55662.130985664531, 30034.828009080142, 33633.52975096587, 58365.369297740915, 39827.669670584852, 399116.16328079719, 81462.203421317012, 66889.496591112664, 54615.010117429432, 27657.463302306252, 147461.179386036, 210849.59536353603, 74270.430130275054, 15185.305677820086, 75602.264220634126, 23809.57178317324, 39908.529073933096, 18671.177990531127, 53658.694475081458, 52219.015934982264, 91144.637927135467, 201465.1238097467, 114923.53912544672, 160064.13463971863, 365434.80798707454, 20549.870581553005, 25907.108995524293, 42227.389069500641, 35334.707866166355, 32934.426740663279, 136610.16727165304, 64165.781133398901, 39655.589472253974, 283693.52937567525, 43929.405439248338, 29827.051773629839, 23119.989512314551, 24158.625386637734, 229600.24299791513, 84025.162940040798, 30538.552155589856, 131262.43110733453, 44944.12755615788, 129069.60503164043, 22051.204328936408, 77720.505626097947, 25694.673934275961, 31186.409782299055, 55791.421944176815, 67710.624137109, 22528.522430858724, 15368.831960516733, 75766.787074908789, 33036.190007471574, 28054.549958593096, 20051.727898766734, 75762.26421662033, 36697.354327064917, 78534.795832478936, 10378.958906073598]
avgWaitingTimeQ1Q2ListBP = [52607.542681004292, 389216.73117256718, 82916.771226141558, 113089.67169002941, 52400.969952254127, 62299.432859906454, 71288.434724807958, 24760.595538226979, 80569.33779864473, 59253.795365935439, 101548.90089207812, 52047.645479844374, 73875.874199711936, 62105.023509992119, 171280.70777054245, 96232.874748405869, 53761.568832901459, 31758.367309839916, 96302.383555451699, 20164.97346698408, 28944.887168562025, 34880.2110090442, 26250.189583076615, 94183.589808140707, 26353.68505121698, 69867.591987676584, 23475.331263511991, 38106.991628449774, 520873.91864662786, 79596.53561720335, 32020.797614082185, 37774.247817548909, 68256.864667161484, 217072.963886051, 89275.868037284381, 171824.4591102972, 47820.266420350177, 56756.872037081594, 52337.258043114867, 42565.898732485031, 64152.809713251198, 33567.39903899101, 34666.000116906682, 58579.12377562208, 47627.444587759644, 392842.29949855339, 85507.003203229833, 222504.90879448826, 62871.058310992434, 28205.185525448967, 140527.69942671462, 200061.22282844401, 75879.819075838779, 24304.831775256327, 90814.472067762428, 24622.1013146183, 47563.223434039355, 25196.8113657924, 56743.950134446772, 77230.259188546028, 97718.721496737257, 206471.45487107604, 116150.84123179331, 164191.55277854923, 362099.46366715903, 21617.999866210659, 30457.758223027762, 46016.256526647856, 37161.872019657647, 44763.84696084007, 139105.32088712984, 67907.859958479108, 38993.806497035825, 297011.07337239163, 51744.098870899594, 34236.282064649939, 30165.14108681208, 26092.737787906884, 232681.29543768195, 84890.700849026893, 43306.46701049323, 135577.44928660686, 161516.21814955154, 136046.86459240396, 24099.991533816858, 79413.759900238685, 27559.16957968516, 33480.725695194255, 60728.872810011548, 70611.593596190651, 27056.427931269718, 28261.357224204978, 79294.226413365643, 35797.634111745843, 30483.417056570957, 22725.468799442628, 77456.472510109612, 43568.114588010729, 81014.254209503037, 13625.823656351033]
avgResponseTime1ListBP = [3854.9706736462449, 36159.381201826029, 5715.7577881684138, 6929.4916793470811, 4648.7023053715229, 3417.7448345954544, 4560.879551698089, 7607.1146293137172, 4301.3290609852411, 4814.3183649816719, 6980.5063159818083, 10833.857409346188, 4596.3420821048176, 17769.602290092444, 7696.7517372532275, 72157.339558388558, 3875.2944855431438, 10789.965437197121, 5151.4438504504988, 3339.7907260493716, 11725.083545926585, 3761.1822265000847, 5157.7037324656467, 4772.9714691560912, 4545.9128499902554, 4025.7220075735968, 10357.324249136815, 11515.381355087904, 5672.4372327153706, 3388.6604986635589, 5858.7874712188777, 5075.5327291978365, 4261.5753101464597, 4182.2824637973708, 10433.494808960349, 12477.86791481846, 4784.507777195081, 10393.436984626504, 15770.166047042023, 14395.306903093002, 12597.880671116571, 6745.5998952250984, 4745.1509008387948, 5075.5051283868925, 12109.818280364627, 13354.824942958978, 3998.2884938943434, 156879.51271504749, 5776.210536901689, 4053.0672586725746, 5587.6437765633646, 4425.9560252276324, 3951.1462089347983, 11787.179207926049, 17748.093344988836, 3677.7720093362868, 6558.2402266767022, 9782.58678863563, 4044.4656690765919, 23134.759232027951, 8006.9253753879275, 5160.0950804706918, 5121.0851191727161, 6162.7781344258237, 5521.4384419696025, 4067.5394419403688, 3482.1721983769517, 10162.433269768482, 3950.6128316208683, 13326.660760748182, 3935.2984551983131, 5088.1629862335922, 4850.1845132551325, 14835.726440634058, 11134.000961468919, 5625.2893546876521, 9675.4649842170184, 3627.1540077396071, 5178.6462102506866, 4630.9483203526725, 16159.947389160192, 6451.1554467379601, 113598.2158894625, 4521.642341034074, 5576.735561297337, 5684.8028448025125, 4359.6159854760908, 5186.260578446464, 5794.8678686692238, 5042.6169070288743, 6298.3686653262266, 14647.2697754281, 4777.069579877696, 5372.3091388517887, 6055.7873114635322, 3749.4564406870645, 5508.7715409054208, 10772.744849259005, 4111.40173632203, 6023.7407559271733]
avgResponseTime2ListBP = [55283.809216010406, 355252.11778322095, 79422.859131141988, 119092.21664252777, 52933.578843482843, 61872.996283751701, 68122.571642379728, 22297.027409638838, 81660.394052403048, 58344.772918454924, 102356.87350826048, 45316.405520934066, 74498.407810689459, 49387.512902235743, 172937.84244555281, 24089.575903234792, 58484.80587907981, 26555.419633841757, 88395.708285444824, 22898.758950903986, 23364.043335075588, 37921.018643266201, 26879.160977433985, 95585.667897664505, 27226.270811382325, 68429.136393890454, 19001.798566706813, 32530.916408052057, 508315.94549933862, 81634.221152014128, 33435.595323301408, 38244.074836986008, 70299.173294159031, 226727.48459498934, 96452.824448858228, 155317.90579915175, 50400.67447235607, 55005.054604319252, 41615.297675436916, 34295.224476024639, 58727.722006606389, 32903.010486709798, 36654.321288655505, 61271.944445955574, 42671.028901025122, 402420.41212502739, 84461.179399084955, 69870.473842360952, 57555.336078565801, 30540.628530607028, 150724.95059081193, 214003.88740523867, 77159.214483687945, 17901.710507131873, 78654.929752268814, 26659.151599127428, 42816.823717233863, 21465.469299283614, 56751.608157104085, 55194.848766659212, 94114.15254023856, 204592.78572809388, 117961.79334672501, 163176.13836065828, 368730.73689741245, 23420.713302784538, 28729.242412522755, 45178.674226729185, 38294.137652560545, 35874.080236361777, 139689.02022413936, 67096.271092081399, 42601.045509799209, 286669.94160631706, 46834.891357869841, 32648.966465259826, 25917.947803940991, 26950.078982033825, 232815.08980579791, 87018.075320219534, 33485.2737689313, 134383.09475163964, 47823.105933611092, 132094.59588926655, 24851.787299821503, 80704.090816857657, 28539.180684241586, 34054.582682936692, 58782.751540662895, 70701.370484822342, 25405.308247944002, 18062.304259519686, 78723.213026321406, 36023.773473535257, 30965.415919674815, 22839.798681857854, 78806.433603185578, 39534.906443713553, 81557.050045781667, 13034.326711981046]
avgResponseTimeQ1Q2ListBP = [58630.083403476317, 395923.27692102181, 88948.302240523786, 118863.11846168383, 57831.535716452447, 67859.933616003036, 77054.37947023203, 30664.532565979025, 86561.521887446099, 65148.23416724991, 107560.07728283507, 57885.408491053611, 79435.996496748194, 68290.088353352199, 177648.65922492062, 103104.60672391881, 59286.456265768429, 37859.125116258321, 101796.27339008221, 25519.168835362998, 34883.746275853875, 40521.888491527461, 31772.976927710399, 99921.413314082354, 32245.123436440805, 75376.006275685824, 29381.399624244037, 44052.830571776329, 527092.34455851954, 85079.50569398138, 37834.155323364161, 43586.641753763819, 74025.972815956207, 222844.97362312599, 94956.043153334031, 177717.16522352555, 53732.207856948611, 62373.86454934191, 58500.972230361585, 48415.429514281604, 70556.573074676882, 39520.632549068265, 40631.356741087249, 64143.181976686225, 53775.457939136868, 399659.22525767214, 91221.095712144597, 229353.11503617564, 69108.226960305983, 33699.84301715358, 146162.8181428285, 205979.31110208668, 81960.093189615189, 30137.609386848668, 96792.518126822833, 30198.257498279752, 53423.044350555989, 31124.189131044881, 62179.049194419567, 83706.163797052635, 103977.56239780177, 212088.85372411823, 122350.22150070501, 169860.41990166699, 368367.06859498267, 27263.142630102036, 35812.323424463008, 52095.06833318749, 42808.756034306694, 50998.987438666707, 144706.1931549838, 73582.930124440274, 44777.380687765966, 303360.48377005453, 57641.480527949192, 39935.961977948042, 36055.766697907769, 31222.093762524848, 238614.17931805196, 90896.894752628839, 49936.040943994485, 141514.58437784945, 168271.7044649673, 141911.69636788327, 29746.863074330722, 85199.423992129436, 33263.158209551293, 39049.911183579294, 66439.819150589843, 76755.056549875459, 32712.656234901693, 34180.127341020154, 85355.145294739224, 41652.386054113049, 36546.250948818306, 28092.336959990433, 82941.000809392383, 49392.503757753744, 86613.146944353444, 19508.663746909737]
q1LengthListBP = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
q2LengthListBP = [0, 325, 0, 0, 2, 2, 0, 0, 28, 0, 2, 5, 7, 0, 23, 0, 0, 0, 0, 0, 20, 0, 0, 0, 2, 1, 5, 0, 101, 35, 7, 0, 4, 194, 563, 0, 2, 59, 39, 4, 3, 5, 67, 2, 0, 2, 6, 0, 0, 0, 520, 1, 0, 0, 153, 1, 0, 0, 166, 4, 1, 0, 4, 158, 1, 0, 0, 39, 13, 0, 74, 1, 0, 0, 0, 1, 0, 0, 0, 44, 0, 0, 6, 62, 0, 3, 0, 0, 0, 10, 0, 2, 0, 1, 2, 0, 141, 4, 3, 0]

fig = plt.figure(1)

ax1 = fig.add_subplot(261, title='Exponential: Average Waiting time for jobs arriving at Q1', xlabel='Average Waiting Time', ylabel='Number of jobs')
ax1.hist(avgWaitingTimeQ1Q2ListExp)


ax2 = fig.add_subplot(262, title='Exponential: Average Waiting time for jobs arriving at Q2', xlabel='Average Waiting Time', ylabel='Number of jobs' )
ax2.hist(avgWaitingTime2ListExp)

ax3 = fig.add_subplot(263, title='Exponential: Average Response time for jobs arriving at Q1', xlabel='Average Response Time', ylabel='Number of jobs')
ax3.hist(avgResponseTimeQ1Q2ListExp)


ax4 = fig.add_subplot(264, title='Exponential: Average Response time for jobs arriving at Q2', xlabel='Average Response Time', ylabel='Number of jobs' )
ax4.hist(avgResponseTime2ListExp)

ax5 = fig.add_subplot(265, title='Exponential: Queue Length at Q1', xlabel='Queue Length', ylabel='Number of jobs')
ax5.hist(q1LengthListExp)


ax6 = fig.add_subplot(266, title='Exponential: Queue Length at Q2', xlabel='Queue Length', ylabel='Number of jobs' )
ax6.hist(q2LengthListExp)

ax7 = fig.add_subplot(267, title='Bounded Pareto: Average Waiting time for jobs arriving at Q1', xlabel='Average Waiting Time', ylabel='Number of jobs')
ax7.hist(avgWaitingTimeQ1Q2ListBP)


ax8 = fig.add_subplot(268, title='Bounded Pareto: Average Waiting time for jobs arriving at Q2', xlabel='Average Waiting Time', ylabel='Number of jobs' )
ax8.hist(avgWaitingTime2ListBP)

ax9 = fig.add_subplot(269, title='Bounded Pareto: Average Response time for jobs arriving at Q1', xlabel='Average Response Time', ylabel='Number of jobs')
ax9.hist(avgResponseTimeQ1Q2ListBP)


#ax10 = fig.add_subplot(2610, title='Bounded Pareto: Average Response time for jobs arriving at Q2', xlabel='Average Response Time', ylabel='Number of jobs' )
#ax10.hist(avgResponseTime2ListBP)

#ax11 = fig.add_subplot(2611, title='Bounded Pareto: Queue Length at Q1', xlabel='Queue Length', ylabel='Number of jobs')
#ax11.hist(q1LengthListBP)


#ax12 = fig.add_subplot(2612, title='Bounded Pareto: Queue Length at Q2', xlabel='Queue Length', ylabel='Number of jobs' )
#ax12.hist(q2LengthListBP)

plt.show()